# Hierarchical series with different levels of granularity

## Hierarchical Time Series Forecasting & Reconciliation
*It has the following steps:*
- Building Hierarchical Time Series
- Hierarchical Forecasting
- Forecast reconciliation
## Time Series at top levels have noticeable trends and seasonality. We built time series models on each time series using ARIMA.
- we took seasonality and trends into consideration when modeling
- we used automatic selection of parameters for ARIMA model
## We tried 3 such simplified reconciliaton strategies: 
- BU, 
- OLS, 
- Mint


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import plotly.express as px


file_path = '/home/olga/ts_year_project/TimeSeriesMasters2023/eda_olga/energy_hourly_dataset_2012.csv'
df = pd.read_csv(file_path)
# df

In [ ]:
#df.set_index('date', inplace=True)

In [ ]:
is_unique = df['date'].duplicated().any()
is_unique

In [ ]:
df.reset_index(inplace=True)

In [ ]:
# fig = px.line(df, x='date', y='value', labels={'value': 'Потребление электроэнергии'}, title='Потребление электроэнергии по часам')
# fig.show()

In [ ]:
# df.index = pd.to_datetime(df.index)

In [ ]:
# df.reset_index(inplace=True)

In [ ]:
# df.info()

In [ ]:
def transform_to_custom_freq(df, freq):
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    # df.set_index('date', inplace=True)
    df_resampled = df.resample(freq).sum()
    # df_resampled.reset_index(inplace=True)
    
    return df_resampled

## week

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [ ]:
# df

In [ ]:
selected_freq = 'W'
df_resampled_week = transform_to_custom_freq(df, selected_freq)
# df_resampled_week

## Daily

In [ ]:
# df

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
# df

In [ ]:
selected_freq = 'D'
df_resampled_day = transform_to_custom_freq(df, selected_freq)
# df_resampled_day

## по месяцам

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [ ]:
# df.reset_index(inplace=True)
selected_freq = 'M'
df_resampled_month = transform_to_custom_freq(df, selected_freq)
# df_resampled_month

In [ ]:
df_resampled_month.reset_index(inplace=True)

In [ ]:
# df_resampled_month

In [ ]:
# fig = px.line(df_resampled_month, x='date', y='value', labels={'value': 'Потребление электроэнергии'}, title='Потребление электроэнергии по месяцам')
# fig.show()

In [ ]:
# df

## Обучение моделей на каждой гранулярности

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# from fbprophet import Prophet

In [ ]:
# !pip install statsmodels
#!python -m pip install prophet

In [ ]:
# !python -m pip install fbprophet

In [ ]:
from pmdarima import auto_arima

In [ ]:
# !pip install pmdarima

## hour

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
train_size = int(len(df) * 0.8)
train_hour, test_hour = df.iloc[:train_size], df.iloc[train_size:]

In [ ]:
auto_arima_model = auto_arima(train_hour['value'], seasonal=True, stepwise=True, suppress_warnings=True)

In [ ]:
auto_arima_model.order

In [ ]:
auto_arima_model.seasonal_order

In [ ]:
print(f'ARIMA Order: {auto_arima_model.order}')
print(f'ARIMA Seasonal Order: {auto_arima_model.seasonal_order}')

In [ ]:
fitted_model = auto_arima_model.fit(train_hour['value'])

In [ ]:
forecast_hour = fitted_model.predict(n_periods=len(test_hour))

In [ ]:
# forecast_hour

In [ ]:
mae = mean_absolute_error(test_hour['value'], forecast_hour)
mse = mean_squared_error(test_hour['value'], forecast_hour)

In [ ]:
print(f'Test Mean Absolute Error: {mae}')
print(f'Test Mean Squared Error: {mse}')

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train_hour.index, train_hour['value'], label='Обучающая выборка')
plt.plot(test_hour.index, test_hour['value'], label='Тестовая выборка')
plt.plot(test_hour.index, forecast_hour, label='Прогноз ARIMA')
plt.title('Прогноз ARIMA с автоматически подобранными параметрами')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend()
# plt.show()

## week

In [ ]:
train_size = int(len(df_resampled_week) * 0.8)
train_week, test_week = df_resampled_week.iloc[:train_size], df_resampled_week.iloc[train_size:]

auto_arima_model = auto_arima(train_week['value'], seasonal=True, stepwise=True, suppress_warnings=True)
print(auto_arima_model.order)
print(auto_arima_model.seasonal_order)

print(f'ARIMA Order: {auto_arima_model.order}')
print(f'ARIMA Seasonal Order: {auto_arima_model.seasonal_order}')

fitted_model = auto_arima_model.fit(train_week['value'])

forecast_week = fitted_model.predict(n_periods=len(test_week))

mae = mean_absolute_error(test_week['value'], forecast_week)
mse = mean_squared_error(test_week['value'], forecast_week)
print(f'Test Mean Absolute Error: {mae}')
print(f'Test Mean Squared Error: {mse}')

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train_week.index, train_week['value'], label='Обучающая выборка')
plt.plot(test_week.index, test_week['value'], label='Тестовая выборка')
plt.plot(test_week.index, forecast_week, label='Прогноз ARIMA')
plt.title('Прогноз ARIMA с автоматически подобранными параметрами')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend()
# plt.show()

In [ ]:
# forecast_week

## day

In [ ]:
train_size = int(len(df_resampled_day) * 0.8)
train_day, test_day = df_resampled_day.iloc[:train_size], df_resampled_day.iloc[train_size:]

auto_arima_model = auto_arima(train_day['value'], seasonal=True, stepwise=True, suppress_warnings=True)
print(auto_arima_model.order)
print(auto_arima_model.seasonal_order)

print(f'ARIMA Order: {auto_arima_model.order}')
print(f'ARIMA Seasonal Order: {auto_arima_model.seasonal_order}')

fitted_model = auto_arima_model.fit(train_day['value'])

forecast_day = fitted_model.predict(n_periods=len(test_day))

mae = mean_absolute_error(test_day['value'], forecast_day)
mse = mean_squared_error(test_day['value'], forecast_day)
print(f'Test Mean Absolute Error: {mae}')
print(f'Test Mean Squared Error: {mse}')


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train_day.index, train_day['value'], label='Обучающая выборка')
plt.plot(test_day.index, test_day['value'], label='Тестовая выборка')
plt.plot(test_day.index, forecast_day, label='Прогноз ARIMA')
plt.title('Прогноз ARIMA с автоматически подобранными параметрами')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.legend()
# plt.show()

In [ ]:
# forecast_day

In [ ]:
# forecast_hour

In [ ]:
# forecast_week

## Применяем реконсиляцию методами bu, ols, mint

In [ ]:
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation

## Bottom-Up

In [ ]:
forecast_day_combined = forecast_hour.resample('D').sum() 
# forecast_day_combined

In [ ]:
mae = mean_absolute_error(test_day['value'], forecast_day_combined)
mse = mean_squared_error(test_day['value'], forecast_day_combined)
print(f'Test Mean Absolute Error: {mae}')
print(f'Test Mean Squared Error: {mse}')

In [ ]:
# Combine lower-level forecasts to obtain higher-level forecasts using the Bottom-Up method
forecast_week_combined = forecast_day_combined.resample('W').sum()  # Aggregating hourly forecasts to weekly
# forecast_week_combined

In [ ]:
mae = mean_absolute_error(test_week['value'], forecast_week_combined)
mse = mean_squared_error(test_week['value'], forecast_week_combined)
print(f'Test Mean Absolute Error: {mae}')
print(f'Test Mean Squared Error: {mse}')

## MinT method

In [ ]:
# tried to take code for categoracal hierarchy from here https://gist.github.com/ngupta23/1e104f18c0440156953ac37dc8e2a616

In [ ]:
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, ERM
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [ ]:
# # You can select a reconciler from our collection
# reconcilers = [
#       BottomUp(),
#       TopDown(method='forecast_proportions'),
#       # TopDown(method='average_proportions'),
#       # TopDown(method='proportion_averages'),
#       MinTrace(method='ols'),
#       # MinTrace(method='wls_var'),
#       # MinTrace(method='mint_shrink'),ls
#       # #ERM(method='reg_bu', lambda_reg=100) # Extremely inneficient
#       ERM(method='closed')
# ]
# hrec = HierarchicalReconciliation(reconcilers=reconcilers)

In [ ]:
# Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, 
#                           Y_df=Y_fitted_df,
#                           S=S_df, tags=tags)
# Y_rec_df.groupby('unique_id').head(FH)

In [ ]:
df_reduced = df.drop(columns=['index', 'id'])
df_reduced

In [ ]:
df_reduced.info()

In [ ]:
df_reduced.reset_index(inplace=True)

In [ ]:
df_reduced

In [ ]:
df_reduced.info()

In [ ]:
df_reduced

In [ ]:
# Добавление столбцов 'hour', 'day' и 'week'
df_reduced['hour'] = df_reduced['date'].astype(str)
df_reduced['day'] = df_reduced['date'].dt.strftime('%Y-%m-%d %H:%M:%S').apply(lambda x: x.split(' ')[0]).astype(str)
df_reduced['week'] = df_reduced['date'].dt.to_period("W-SUN").apply(lambda x: x.start_time).astype(str)
df_reduced['month'] = df_reduced['date'].dt.strftime('%Y-%m').astype(str)

# Вывод результата
df_reduced

In [ ]:
df_reduced['date'] = df_reduced['month'].astype(str)

In [ ]:
df_reduced.info()

In [ ]:
df_reduced = df_reduced.rename(columns={'value':'y', 'date':'ds'})
df_reduced

In [ ]:
# Create hierarchical structure and constraints
hierarchy_levels = [
    ['month'],
    ['month', 'week'],
    ['month', 'week', 'day'],
    ['month', 'week', 'day', 'hour']
  ]
Y_hier_df, S_df, tags = aggregate(df=df_reduced, spec=hierarchy_levels)
Y_hier_df = Y_hier_df.reset_index()
print('S_df.shape', S_df.shape)
print('Y_hier_df.shape', Y_hier_df.shape)
print("tags['month']", tags['month'])

In [ ]:
Y_hier_df["unique_id"].unique()

In [ ]:
S_df

In [ ]:
# import plotly.graph_objects as go
# 
# # Ваш код: создание данных S_df и tags
# 
# # Создание иерархического графика
# fig = go.Figure(data=go.Sankey(
#     node = dict(
#       label = tags
#     ),
#     link = dict(
#       source = S_df['month'], # индекс первоначальной ноды
#       target = S_df['hour'], # индекс целевой ноды
#       value = S_df['value'] # значение связи
#   )))
# 
# fig.show()

In [ ]:
FH = 2  # Forecast Horizon

In [ ]:
Y_hier_df

In [ ]:
Y_test_df

In [ ]:
# Split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(FH)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

Y_train_df.info(), Y_test_df.info()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(Y_hier_df, test_size=0.2, random_state=42)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
metric = "mse"
from statsforecast.core import StatsForecast
from tqdm.autonotebook import tqdm
import random
import os
from datasetsforecast.hierarchical import HierarchicalData
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import  BottomUp, TopDown, MiddleOut, MinTrace, ERM
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from sklearn.metrics import mean_squared_error as mse

In [ ]:
# !pip install statsforecast
# !pip install datasetsforecast

In [ ]:
# Compute base auto-ARIMA predictions
fcst = StatsForecast(df = train_df, models=[AutoARIMA(season_length= 7)], freq='D', n_jobs=-1)
x_hat = fcst.forecast(h = 7)

In [ ]:
x_hat

In [ ]:
test_df

In [ ]:
test_df.set_index('unique_id')

In [ ]:
test_df.info()

In [ ]:
test_df['ds'] = pd.to_datetime(test_df['ds'])
test_df.info()

In [ ]:
test_df

In [ ]:
x_hat.info()

In [ ]:
xmat = pd.merge(left = test_df, right = x_hat, on = ['ds', 'unique_id'])
xmat.head(3)

## Сравниваем результаты base и после реконсиляции

In [ ]:
# !pip install hierarchicalforecast

## Выводы

- Hierarchical Forecasting & Reconciliation should improve the overall prediction of intermittent time series with correct reconciliation strategy
- Hierarchical Forecasting & Reconciliation can apply to any time series with a hierarchy structure

## HTS offers a lot of flexibilities:
- Assist to build hierarchical time series from every level data
- We can choose different ML algorithms for different levels of hierarchical time series
- 3 reconciliation strategies are available at this moment.
